In [40]:
import tensorflow as tf
import scipy.io as sio
import numpy as np
import time
from datetime import datetime
import os
import h5py as h5
import utils
import tf_utils
import parser_ops
import masks.ssdu_masks as ssdu_masks
import UnrollNet

In [47]:
file_name = '/srv/share4/ksarangmath3/mri/data/singlecoil_train/file1000017.h5'


In [48]:
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [49]:
f = h5.File(file_name, "r")

In [52]:
kspace_train = f['kspace'][:]
# kspace_train = np.expand_dims(kspace_train, axis=1)
kspace_shape = kspace_train.shape

In [54]:
np.expand_dims(kspace_train,axis=3).shape

(34, 640, 372, 1)

In [29]:
rec_rss = f['reconstruction_rss'][:]
rec_rss.shape

(34, 320, 320)

In [30]:
# Ground truth
rec_esc = f['reconstruction_esc'][:]
rec_esc.shape

(34, 320, 320)

In [31]:
from fastmri.data.subsample import create_mask_for_mask_type

In [32]:
mask_fn = create_mask_for_mask_type("random",[0.08], [4])

In [33]:
mask = mask_fn(kspace_shape, 1)

In [34]:
mask.shape

torch.Size([1, 1, 640, 1])

In [35]:
kspace_shape

(34, 1, 640, 372)

In [36]:
sens_maps = bart.bart(1, f"ecalib -d0 -m1 -r", kspace_train)

NameError: name 'bart' is not defined